In [5]:
# -*- coding: utf-8 -*-
import json, os
import numpy as np
import pandas as pd
from collections import defaultdict, deque

# ================= Eingaben / Pfade ============================================
graph_json_path = r"C:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\exploratory\graph\bspMitReaktanz.json"
csv_folder      = r"C:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\exploratory\handling_graph\out\nodes"
ts_col = "timestamp"
p_col  = "P_Datapoint_ID"

S_base_MVA   = 100.0
V_kV_default = 110.0
COSPHI_MIN   = 0.95          # konservativ für Strom-/Auslastungsberechnung

# Knotentypen ohne Messwerte (werden 0 MW gesetzt)
NO_MEASURE_TYPES = {"Sammelschiene", "Leitungsknoten", "junction", "busbar"}

# Behandlung fehlender Reaktanzen:
CONTRACT_ZERO_X = True
FILL_DEFAULT_X  = False
DEFAULT_X_PER_KM = {110.0: 0.35, 220.0: 0.25, 380.0: 0.20}

# Slack-Heuristik
PREFER_TYPES_FOR_SLACK = {"uw_field", "substation", "busbar", "Sammelschiene"}
AVOID_TYPES_FOR_SLACK  = {"BESS", "battery", "load"}

# ================== HELFER ======================================================
def edge_voltage_kV(u, v, efeat, node_features, V_kV_default):
    if efeat.get("Voltage_kV") not in (None, ""):
        try: return float(efeat["Voltage_kV"])
        except: pass
    uV = node_features.get(u, {}).get("Voltage_kV", None)
    vV = node_features.get(v, {}).get("Voltage_kV", None)
    try: uV = float(uV) if uV is not None else None
    except: uV = None
    try: vV = float(vV) if vV is not None else None
    except: vV = None
    if (uV is not None) and (vV is not None) and abs(uV - vV) < 1e-6:
        return uV
    return V_kV_default

def edge_raw_X_ohm(efeat):
    if efeat.get("X_total_ohm") not in (None, ""):
        return float(efeat["X_total_ohm"])
    xpk = efeat.get("X_ohm_per_km", None)
    Lkm = efeat.get("length_km", None)
    try: xpk = float(xpk) if xpk is not None else None
    except: xpk = None
    try: Lkm = float(Lkm) if Lkm is not None else None
    except: Lkm = None
    if (xpk is not None) and (Lkm is not None):
        return xpk * Lkm
    return None

def ohm_to_pu(X_ohm, V_kV, S_base_MVA):
    Zb = (V_kV*1e3)**2 / (S_base_MVA*1e6)
    return X_ohm / Zb

def mw_to_amp_with_pf(P_MW, V_kV, cosphi_min=COSPHI_MIN):
    if V_kV is None or V_kV <= 0: V_kV = V_kV_default
    if cosphi_min <= 0 or cosphi_min > 1:
        raise ValueError("cosφ_min in (0,1] erwartet")
    P_W = abs(P_MW) * 1e6
    U_V = V_kV * 1e3
    return P_W / (np.sqrt(3.0) * U_V * cosphi_min)

class UF:
    def __init__(self, items):
        self.parent = {x:x for x in items}
        self.rank   = {x:0 for x in items}
    def find(self, x):
        p = self.parent[x]
        if p != x:
            self.parent[x] = self.find(p)
        return self.parent[x]
    def union(self, a,b):
        ra, rb = self.find(a), self.find(b)
        if ra == rb: return
        if self.rank[ra] < self.rank[rb]:
            self.parent[ra] = rb
        elif self.rank[ra] > self.rank[rb]:
            self.parent[rb] = ra
        else:
            self.parent[rb] = ra
            self.rank[ra] += 1

def connected_components(nodes, lines):
    idx = {n:i for i,n in enumerate(nodes)}
    adj = [[] for _ in nodes]
    for (u,v,*_) in lines:
        ui, vi = idx[u], idx[v]
        adj[ui].append(vi); adj[vi].append(ui)
    seen = [False]*len(nodes)
    comps = []
    for i in range(len(nodes)):
        if seen[i]: continue
        q = deque([i]); seen[i]=True; comp=[]
        while q:
            x=q.popleft(); comp.append(nodes[x])
            for y in adj[x]:
                if not seen[y]:
                    seen[y]=True; q.append(y)
        comps.append(comp)
    return comps

def choose_slack_for_component(component_nodes, lines, node_types):
    deg = {n:0 for n in component_nodes}
    for (u,v,*_) in lines:
        if u in deg and v in deg:
            deg[u]+=1; deg[v]+=1
    good = [n for n in component_nodes if node_types.get(n,"") in PREFER_TYPES_FOR_SLACK]
    if good:
        return max(good, key=lambda n: deg[n])
    return max(component_nodes, key=lambda n: deg[n])

def pretty_matrix(name, M, rows=None, cols=None, fmt="{: .6f}", maxn=8):
    r, c = M.shape
    rr = rows or [f"r{i}" for i in range(r)]
    cc = cols or [f"c{j}" for j in range(c)]
    print(f"\n--- {name}  [{r}x{c}] ---")
    take_r = min(r, maxn)
    take_c = min(c, maxn)
    header = ["{:>12}".format("")] + ["{:>12}".format(cc[j]) for j in range(take_c)]
    print(" ".join(header))
    for i in range(take_r):
        row = ["{:>12}".format(rr[i])] + ["{:>12}".format(fmt.format(M[i,j])) for j in range(take_c)]
        print(" ".join(row))
    if r>take_r or c>take_c:
        print("...")

# ================== GRAPH LADEN ================================================
with open(graph_json_path, "r", encoding="utf-8") as f:
    graph = json.load(f)

nodes, edges = [], []
node_types, node_features = {}, {}
for it in graph:
    if "data" not in it: continue
    d = it["data"]
    if "source" in d and "target" in d:
        edges.append(d)
    else:
        nid = d["id"]
        nodes.append(nid)
        node_types[nid]    = (d.get("type") or "").strip()
        node_features[nid] = d.get("features", {}) or {}

if not nodes:
    raise RuntimeError("Keine Knoten im Graph gefunden.")

print("=== Originalknoten (n={}) ===".format(len(nodes)))
print(nodes)

# ================== KANTEN-PREPROCESSING (Zero-X behandeln) ====================
uf = UF(nodes)
electrical_edges = []   # (u,v,feat, VkV, X_ohm, tag)
visual_to_contract = [] # (u,v) ohne brauchbare X

for e in edges:
    u, v = e["source"], e["target"]
    feat = e.get("features", {}) or {}
    VkV  = edge_voltage_kV(u, v, feat, node_features, V_kV_default)
    Xohm = edge_raw_X_ohm(feat)

    if Xohm is None or Xohm <= 0.0:
        if FILL_DEFAULT_X:
            Lkm = feat.get("length_km", None)
            try: Lkm = float(Lkm) if Lkm is not None else None
            except: Lkm = None
            xpk = DEFAULT_X_PER_KM.get(VkV, None)
            if (Lkm is not None) and (xpk is not None) and Lkm > 0 and xpk > 0:
                Xohm = xpk * Lkm
                electrical_edges.append((u, v, feat, VkV, Xohm, "estimated"))
            else:
                if CONTRACT_ZERO_X:
                    visual_to_contract.append((u, v))
        else:
            if CONTRACT_ZERO_X:
                visual_to_contract.append((u, v))
    else:
        electrical_edges.append((u, v, feat, VkV, Xohm, "given"))

for (u, v) in visual_to_contract:
    if u in node_types and v in node_types:
        uf.union(u, v)

rep_map = {nid: uf.find(nid) for nid in nodes}
super_nodes = sorted(set(rep_map.values()))

members_by_rep = defaultdict(list)
for nid, rep in rep_map.items():
    members_by_rep[rep].append(nid)

def merged_type(members):
    types = [node_types.get(m, "") for m in members if node_types.get(m, "")]
    for t in types:
        if t.lower() in {"bess", "battery"}:
            return "BESS"
    return types[0] if types else ""

super_node_types    = {rep: merged_type(members_by_rep[rep]) for rep in super_nodes}
super_node_features = {rep: {} for rep in super_nodes}

super_edges = []  # (ru, rv, eid, feat, VkV, X_ohm)
for (u, v, feat, VkV, Xohm, tag) in electrical_edges:
    ru, rv = rep_map[u], rep_map[v]
    if ru == rv:
        continue
    eid = feat.get("id", f"{u}-{v}")
    super_edges.append((ru, rv, eid, feat, VkV, Xohm))

if not super_edges:
    raise RuntimeError("Nach Kontraktion/Filterung keine elektrischen Kanten übrig.")

print("\n=== Superknoten (n={}) ===".format(len(super_nodes)))
print(super_nodes)
print("Superknoten-Mitglieder:")
for rep, mem in members_by_rep.items():
    if rep in super_nodes:
        print(f"  {rep}: {mem}")

print("\nSuperknoten-Typen:")
for rep in super_nodes:
    print(f"  {rep}: {super_node_types.get(rep,'')}")

print("\nSuperkanten ({}):".format(len(super_edges)))
for (ru, rv, eid, feat, VkV, Xohm) in super_edges:
    print(f"  {eid}: {ru} -> {rv}, VkV≈{VkV}, XΩ≈{Xohm}")

# ================== CSV LADEN & AUF SUPERKNOTEN AGGREGIEREN ====================
def load_series_for_node(nid):
    path = os.path.join(csv_folder, f"{nid}.csv")
    if not os.path.isfile(path):
        return None
    df = pd.read_csv(path)
    if ts_col not in df.columns:
        raise ValueError(f"Zeitspalte '{ts_col}' fehlt in {path}")
    dt = pd.to_datetime(df[ts_col], errors="coerce", utc=True)
    dt = dt.dt.tz_localize(None)  # tz-naiv
    mask = ~dt.isna()
    if not mask.any():
        raise ValueError(f"Alle Zeitstempel in {path} sind ungültig.")
    df = df.loc[mask].copy()
    dt = dt.loc[mask]
    df.index = dt
    df.drop(columns=[ts_col], inplace=True, errors="ignore")
    if p_col not in df.columns:
        raise ValueError(f"Leistungsspalte '{p_col}' fehlt in {path}")
    s = pd.to_numeric(df[p_col], errors="coerce").fillna(0.0)
    s.index.name = None
    s.name = nid
    s = s.sort_index()
    if not isinstance(s.index, pd.DatetimeIndex):
        raise TypeError(f"{path}: Index ist kein DatetimeIndex nach Parsing.")
    return s

series_raw = {}
for nid in nodes:
    if node_types.get(nid, "") in NO_MEASURE_TYPES:
        series_raw[nid] = None
    else:
        series_raw[nid] = load_series_for_node(nid)

all_series = [s for s in series_raw.values() if s is not None]
if not all_series:
    raise RuntimeError("Keine Messreihen gefunden – prüfe csv_folder / Dateinamen / Spalten.")

print("\n=== Test Zeitachsen ===")
for nid, s in series_raw.items():
    if s is not None:
        print(f"{nid}: len={len(s)}, idx[0]={s.index[0]} ... tz={s.index.tz}")

P_df = pd.concat(all_series, axis=1).sort_index().fillna(0.0)  # MW
for nid in nodes:
    if nid not in P_df.columns:
        P_df[nid] = 0.0

# Superknoten-Leistung = Summe der Mitgliedsleistungen
for rep, members in members_by_rep.items():
    cols = [c for c in members if c in P_df.columns]
    P_df[rep] = P_df[cols].sum(axis=1) if cols else 0.0

P_df = P_df[super_nodes]
print("\n=== P_df Spalten (Superknoten) ===")
print(list(P_df.columns))
print("Zeilen (Zeitstempel):", len(P_df))

# ================== B-MATRIX (Supernetz, p.u.) =================================
n = len(super_nodes)
node_index = {nid:i for i, nid in enumerate(super_nodes)}
B = np.zeros((n, n), dtype=float)

lines = []  # (ru, rv, X_pu, eid, feat, VkV)
for (ru, rv, eid, feat, VkV, Xohm) in super_edges:
    X_pu = ohm_to_pu(Xohm, VkV, S_base_MVA)
    if X_pu <= 0: continue
    lines.append((ru, rv, X_pu, eid, feat, VkV))

if not lines:
    raise RuntimeError("Keine gültigen Leitungen mit X_pu > 0 im Supernetz.")

for (ru, rv, X_pu, eid, feat, VkV) in lines:
    i, j = node_index[ru], node_index[rv]
    b = 1.0 / X_pu
    B[i, j] -= b
    B[j, i] -= b
for i in range(n):
    B[i, i] = -np.sum(B[i, :])

pretty_matrix("B (nodale Suszeptanz in p.u.)", B, rows=super_nodes, cols=super_nodes)

# ================== KOMPONENTEN & SLACK JE KOMPONENTE ==========================
components = connected_components(super_nodes, lines)
if not components:
    raise RuntimeError("Keine Komponenten im Supernetz gefunden.")
print("\n=== Komponenten ===")
for comp in components:
    print("  ", comp)

slack_by_comp = {}
for comp in components:
    slack = choose_slack_for_component(comp, lines, super_node_types)
    slack_by_comp[frozenset(comp)] = slack

node_idx = {n:i for i, n in enumerate(super_nodes)}
comp_data = {}  # comp_key -> dict

for comp in components:
    comp_key = frozenset(comp)
    slack = slack_by_comp[comp_key]
    comp_idx = [node_idx[n] for n in comp]
    B_sub = B[np.ix_(comp_idx, comp_idx)]

    non_slack_nodes_local = [n for n in comp if n != slack]
    non_slack_idx_global  = [node_idx[n] for n in non_slack_nodes_local]

    comp_to_local = {node_idx[n]: i for i, n in enumerate(comp)}
    mask_local = [comp_to_local[node_idx[n]] for n in non_slack_nodes_local]

    B_rr = B_sub[np.ix_(mask_local, mask_local)]
    try:
        B_rr_inv = np.linalg.inv(B_rr)
    except np.linalg.LinAlgError:
        raise RuntimeError(f"B_rr in Komponente {list(comp)} ist singulär. Prüfe Zero-X-Kontraktion / Netzanschlüsse.")

    # Konditionszahl als Qualitätsindikator
    cond = np.linalg.cond(B_rr)

    comp_data[comp_key] = {
        "slack": slack,
        "B_rr": B_rr, "B_rr_inv": B_rr_inv, "cond": cond,
        "non_slack_nodes": non_slack_nodes_local,
        "non_slack_idx": non_slack_idx_global,
        "comp_nodes": comp,
    }
    pretty_matrix(f"B_rr (Komponente {list(comp)} | Slack={slack} | cond={cond: .2e})",
                  B_rr, rows=non_slack_nodes_local, cols=non_slack_nodes_local)

# ================== PTDF JE KOMPONENTE (zeitinvariant) =========================
PTDF_by_comp = {}  # comp_key -> {"PTDF":..., "lines":..., "line_ids":[...]}

for comp in components:
    comp_key = frozenset(comp)
    info = comp_data[comp_key]
    slack = info["slack"]
    non_slack_nodes_local = info["non_slack_nodes"]
    B_rr_inv = info["B_rr_inv"]

    comp_lines = [(u, v, Xpu, eid, feat, VkV)
                  for (u, v, Xpu, eid, feat, VkV) in lines
                  if (u in comp and v in comp)]
    m = len(comp_lines); k = len(non_slack_nodes_local)
    if m == 0:
        PTDF_by_comp[comp_key] = {"PTDF": np.zeros((0, k)), "lines": [], "line_ids": []}
        continue

    A_r = np.zeros((m, k), dtype=float)
    line_ids = []
    for ell, (u, v, Xpu, eid, feat, VkV) in enumerate(comp_lines):
        line_ids.append(eid)
        if u != slack:
            A_r[ell, non_slack_nodes_local.index(u)] = +1.0
        if v != slack:
            A_r[ell, non_slack_nodes_local.index(v)] = -1.0

    B_ell = np.diag([1.0 / Xpu for (_, _, Xpu, _, _, _) in comp_lines])
    PTDF = B_ell @ A_r @ B_rr_inv  # (m x k)
    PTDF_by_comp[comp_key] = {"PTDF": PTDF, "lines": comp_lines, "line_ids": line_ids}

    # Ausgabe
    print(f"\n=== PTDF (Komponente {list(comp)} | Slack={slack})  shape={PTDF.shape} ===")
    # Erste Zeilen zeigen
    pretty_matrix("A_r", A_r, rows=line_ids, cols=non_slack_nodes_local)
    pretty_matrix("B_ell", B_ell, rows=line_ids, cols=line_ids)
    pretty_matrix("PTDF (MW/MW)", PTDF, rows=line_ids, cols=non_slack_nodes_local)

    # Numerischer Gegencheck: Δf ≈ PTDF·ΔP
    if k > 0:
        test_node = non_slack_nodes_local[0]
        j = non_slack_nodes_local.index(test_node)
        dP = 1.0  # MW
        print(f"\n--- Finite-Difference-Check für Node={test_node}, ΔP={dP} MW ---")
        print("PTDF·ΔP (erwartete Δf je Leitung) :")
        est = PTDF[:, j] * dP
        for lid, val in zip(line_ids, est):
            print(f"  {lid:>25s}: {val:+.6f} MW")

# ================== ZEITREIHEN: DC-LASTFLUSS JE ZEITPUNKT ======================
edge_ids_global = [e[3] for e in lines]
flows_MW_ts, angles_ts = [], []

# Hilfs-Lookups für Spannung/Limit je Leitung
edge_meta = {}
for (u, v, Xpu, eid, feat, VkV) in lines:
    edge_meta[eid] = {"u":u, "v":v, "Xpu":Xpu, "VkV":VkV, "limit_A": feat.get("Strom_Limit_in_A", None)}

for t, row in P_df.iterrows():
    theta_global = np.zeros(len(super_nodes))
    flows_now = {}

    for comp in components:
        comp_key = frozenset(comp)
        info = comp_data[comp_key]
        slack = info["slack"]
        non_slack_nodes_local = info["non_slack_nodes"]
        non_slack_idx_global  = info["non_slack_idx"]
        B_rr_inv = info["B_rr_inv"]

        # Bilanz in der Komponente prüfen & Slack angleichen
        P_comp = np.array([row[n] for n in comp], dtype=float)
        mismatch = P_comp.sum()
        if abs(mismatch) > 1e-9:
            row.loc[slack] = row.get(slack, 0.0) - mismatch

        # Nicht-Slack P in p.u.
        P_ns_MW = np.array([row[n] for n in non_slack_nodes_local], dtype=float)
        P_ns_pu = P_ns_MW / S_base_MVA

        # Winkel
        theta_ns = B_rr_inv @ P_ns_pu
        for local_idx, idx_g in enumerate(non_slack_idx_global):
            theta_global[idx_g] = theta_ns[local_idx]
        # Slack bleibt 0

        # Flüsse (nur Leitungen dieser Komponente)
        comp_lines = PTDF_by_comp[comp_key]["lines"]
        for (u, v, Xpu, eid, feat, VkV) in comp_lines:
            i, j = node_index[u], node_index[v]
            f_pu = (theta_global[i] - theta_global[j]) / Xpu
            f_MW = f_pu * S_base_MVA
            flows_now[eid] = f_MW

    flows_MW_ts.append(pd.Series(flows_now, name=t))
    angles_ts.append(pd.Series(theta_global, index=super_nodes, name=t))

flows_MW_df = pd.DataFrame(flows_MW_ts).sort_index()
angles_df   = pd.DataFrame(angles_ts).sort_index()

print("\n=== Beispielausgabe: erster Zeitstempel ===")
if len(flows_MW_df) > 0:
    t0 = flows_MW_df.index[0]
    print("t0 =", t0)

    # 1) P @ t0 (MW) – Bilanz
    P0 = P_df.loc[t0, super_nodes]
    print("\nP (MW) @ t0:")
    print(P0.to_dict())
    print("Bilanz ΣP (MW) der Komponente(n) wird via Slack ausgeglichen.")

    # 2) Winkel @ t0
    print("\nWinkel θ (rad) @ t0:")
    print(angles_df.loc[t0].to_dict())

    # 3) Flüsse @ t0
    print("\nLeitungsflüsse (MW) @ t0:")
    print(flows_MW_df.loc[t0].to_dict())

    # 4) Knotenbilanz aus Flüssen rekonstruieren (Check)
    recon = defaultdict(float)
    for eid, fMW in flows_MW_df.loc[t0].items():
        u = edge_meta[eid]["u"]; v = edge_meta[eid]["v"]
        recon[u] += fMW
        recon[v] -= fMW
    recon = dict(recon)
    print("\nRekonstruierte Knoteneinspeisungen aus Flüssen (MW) @ t0:")
    print(recon)
    print("Summe rekonstruierter P (MW):", sum(recon.values()))

# ================== AUSLASTUNG (Leitungen & Felder) @ t0 =======================
if len(flows_MW_df) > 0:
    t0 = flows_MW_df.index[0]
    line_rows = []
    for eid, fMW in flows_MW_df.loc[t0].items():
        meta = edge_meta[eid]
        VkV  = meta["VkV"]
        I_A  = mw_to_amp_with_pf(fMW, VkV, COSPHI_MIN)
        limA = meta["limit_A"]
        util = None
        if limA not in (None, ""):
            try:
                limA = float(limA)
                if limA > 0:
                    util = abs(I_A)/limA*100.0
            except:
                limA = None
        line_rows.append({
            "edge_id": eid, "u": meta["u"], "v": meta["v"], "U_kV": VkV,
            "P_MW": fMW, "I_A": I_A, "limit_A": limA, "util_%": util
        })
    line_df = pd.DataFrame(line_rows).set_index("edge_id")
    line_df_sorted = line_df.sort_values(by=["util_%"], ascending=False)
    print("\n=== Leitungsauslastung @ t0 (mit cosφ_min={}) ===".format(COSPHI_MIN))
    print(line_df_sorted.round(3).to_string())

    # Feld-/Knoten-Auslastung: max(|I|) der anliegenden Leitungen pro Knoten
    node_rows = []
    # Map (Knoten)->Liste|I|
    node_I = defaultdict(list)
    for eid, row in line_df.iterrows():
        u = row["u"]; v = row["v"]; Iabs = abs(row["I_A"]); limA = edge_meta[eid]["limit_A"]
        node_I[u].append(Iabs); node_I[v].append(Iabs)
    for n in super_nodes:
        maxI = max(node_I.get(n, [0.0])) if node_I.get(n) else 0.0
        limA = super_node_features.get(n,{}).get("Strom_Limit_in_A", None)  # nur falls gepflegt
        util = None
        if limA not in (None, ""):
            try:
                limA = float(limA)
                if limA > 0: util = maxI/limA*100.0
            except:
                limA = None
        node_rows.append({"node": n, "type": super_node_types.get(n,""), "I_max_A": maxI, "limit_A": limA, "util_%": util})
    node_df = pd.DataFrame(node_rows).set_index("node").sort_values(by=["util_%"], ascending=False)
    print("\n=== Feld-/Knoten-Auslastung @ t0 (max. anliegender Strom) ===")
    print(node_df.round(3).to_string())

# ================== ZUSAMMENFASSUNG ===========================================
print("\n=== Zusammenfassung ===")
print(f"Superknoten (n={len(super_nodes)}): {super_nodes}")
print(f"Anzahl elektrischer Leitungen (nach Kontraktion): {len(lines)}")
print(f"Anzahl Zeitstempel: {len(flows_MW_df)}")
print("Komponenten & Slack:")
for comp in components:
    ck = frozenset(comp)
    info = comp_data[ck]
    print(f" - {list(comp)} | Slack: {info['slack']} | kond(B_rr)≈{info['cond']:.2e}")


=== Originalknoten (n=11) ===
['SHUW', 'SHUW_E24', 'JUBO_A5', 'JUBO_E01', 'JUBO_E03', 'BOLS_A5', 'BOLS_E42', 'Tarp', 'Tarp_E01', 'JUBO_E02', 'JUBO']

=== Superknoten (n=6) ===
['BOLS_A5', 'BOLS_E42', 'JUBO', 'JUBO_A5', 'SHUW', 'Tarp']
Superknoten-Mitglieder:
  SHUW: ['SHUW', 'SHUW_E24']
  JUBO_A5: ['JUBO_A5']
  JUBO: ['JUBO_E01', 'JUBO_E03', 'JUBO_E02', 'JUBO']
  BOLS_A5: ['BOLS_A5']
  BOLS_E42: ['BOLS_E42']
  Tarp: ['Tarp', 'Tarp_E01']

Superknoten-Typen:
  BOLS_A5: junction
  BOLS_E42: BESS
  JUBO: uw_field
  JUBO_A5: junction
  SHUW: busbar
  Tarp: busbar

Superkanten (5):
  SHUW_E24-JUBO_A5: SHUW -> JUBO_A5, VkV≈110.0, XΩ≈0.6344
  JUBO_A5-JUBO_E01: JUBO_A5 -> JUBO, VkV≈110.0, XΩ≈1.402246
  BOLS_A5-JUBO_A5: BOLS_A5 -> JUBO_A5, VkV≈110.0, XΩ≈2.133054
  BOLS_A5-BOLS_E42: BOLS_A5 -> BOLS_E42, VkV≈110.0, XΩ≈0.0039900000000000005
  Tarp_E01-BOLS_A5: Tarp -> BOLS_A5, VkV≈110.0, XΩ≈5.25084

=== Test Zeitachsen ===
SHUW_E24: len=1344, idx[0]=2025-10-07 05:15:00 ... tz=None
JUBO_E01: len=134